## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,31.66,93,75,20.71,light snow
1,1,Vaini,TO,-21.2000,-175.2000,77.16,73,75,6.91,broken clouds
2,2,Katsuura,JP,35.1333,140.3000,70.11,80,92,6.42,overcast clouds
3,3,Palmer,US,42.1584,-72.3287,71.42,44,40,1.99,scattered clouds
4,4,Hermanus,ZA,-34.4187,19.2345,60.91,79,42,4.81,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,77.16,73,75,6.91,broken clouds
9,9,Kailua,US,21.4022,-157.7394,83.14,74,40,8.05,scattered clouds
10,10,Atuona,PF,-9.8000,-139.0333,80.29,75,14,17.60,few clouds
12,12,Calabozo,VE,8.9242,-67.4293,85.57,59,89,4.50,overcast clouds
14,14,Dingle,PH,10.9995,122.6711,77.72,85,66,4.14,broken clouds
16,16,Omboue,GA,-1.5746,9.2618,77.45,93,94,6.06,overcast clouds
19,19,Rikitea,PF,-23.1203,-134.9692,77.05,79,96,18.97,light rain
20,20,Hithadhoo,MV,-0.6000,73.0833,83.89,72,100,16.06,overcast clouds
22,22,Sitio Novo Do Tocantins,BR,-5.5183,-47.4611,78.94,75,66,2.64,broken clouds
26,26,Aransas Pass,US,27.9095,-97.1500,83.01,85,40,13.00,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
9,Kailua,US,83.14,scattered clouds,21.4022,-157.7394,
10,Atuona,PF,80.29,few clouds,-9.8000,-139.0333,
12,Calabozo,VE,85.57,overcast clouds,8.9242,-67.4293,
14,Dingle,PH,77.72,broken clouds,10.9995,122.6711,
16,Omboue,GA,77.45,overcast clouds,-1.5746,9.2618,
19,Rikitea,PF,77.05,light rain,-23.1203,-134.9692,
20,Hithadhoo,MV,83.89,overcast clouds,-0.6000,73.0833,
22,Sitio Novo Do Tocantins,BR,78.94,broken clouds,-5.5183,-47.4611,
26,Aransas Pass,US,83.01,scattered clouds,27.9095,-97.1500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
indexNames = hotel_df[hotel_df["Hotel Name"] == ""].index
clean_hotel_df = hotel_df.drop(indexNames)
print(len(clean_hotel_df))

232
220


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))